# Multilayer Perceptron (MLP)

## Libraries

Load the libraries.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from keras import regularizers
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import label_binarize
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from imblearn.under_sampling import RandomUnderSampler

## Data Preprocessing

Load the data.

In [ ]:
data = pd.read_csv("data/000webhost_subset_classifed_featureExtracted.csv", error_bad_lines=False)

Split the data into traning and testing sets.

In [ ]:
X = data.drop(columns=["password", "strength", "length", "uppercase", "lowercase", "digits", "special", "consecutive_char_type_count", "cracking_time"])
y = data["strength"]

rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

# First, split the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Then, split the training set again to create a validation set.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

Categorize the output.

In [ ]:
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_val_encoded = encoder.fit_transform(y_val)
y_test_encoded = encoder.fit_transform(y_test)

## Model Construction and Evaluation

Build the model.

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=(X_train.shape[1], )),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.01)),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(16, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(4, activation='softmax')
    ]
)

model.summary()

Compile and fit the model. 

In [ ]:
batch_size = 32
epochs = 50

# Create the Adam optimizer with the desired learning rate
optimizer = Adam(learning_rate=0.0005)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
training_history = model.fit(X_train, y_train_encoded, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Run the model on the validation set. 

In [ ]:
val_loss, val_acc = model.evaluate(X_val, y_val_encoded)
print(f"Test accuracy: {val_acc:.4f}")

Save the model. 

In [ ]:
model.save("mlp_model.h5")

# Analysis of the Model

Load the model. 

In [ ]:
loaded_model = load_model("mlp_model.h5")

Create a diagram of the model.

In [ ]:
plot_model(loaded_model, to_file='MLP.png', show_shapes=True, show_layer_names=True)

Test the model.

In [ ]:
test_loss, test_acc = loaded_model.evaluate(X_test, y_test_encoded)
print(f"Test accuracy: {test_acc:.4f}")


Predict on the test set.

In [ ]:
y_pred = loaded_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

Calculate different performance metrics.

In [ ]:
precision = precision_score(y_test_encoded, y_pred_classes, average='weighted')
recall = recall_score(y_test_encoded, y_pred_classes, average='weighted')
f1 = f1_score(y_test_encoded, y_pred_classes, average='weighted')

Print the metrics.

In [ ]:
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print(classification_report(y_test_encoded, y_pred_classes))


Find the ROC and AUC.

In [ ]:
n_classes = 4
y_test_binarized = label_binarize(y_test_encoded, classes=[0, 1, 2, 3])
fpr, tpr, roc_auc = dict(), dict(), dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

Plot the ROC curve.

In [ ]:
# Calculate the macro-average AUC
auc_macro = sum(roc_auc.values()) / len(roc_auc)
print("Macro-average AUC:", auc_macro)

# Plot the ROC curves with AUC values
plt.figure()
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('MLP ROC')
plt.legend(loc="lower right")
plt.savefig("MLP ROC", dpi=300, bbox_inches='tight')
plt.show()


Rank the features by importance.

In [ ]:
# Get the absolute values of the weights in the input layer
mlp_weights_abs = np.abs(loaded_model.layers[0].get_weights()[0])

# Sum the weights across neurons
mlp_feature_importance = np.sum(mlp_weights_abs, axis=1)

# Rank the features by their importance
mlp_feature_rank = np.argsort(mlp_feature_importance)[::-1]

print("MLP feature importance ranking:")
for i, rank in enumerate(mlp_feature_rank):
    print(f"Feature {rank}: {mlp_feature_importance[rank]:.4f}")


Plot the history. 

In [ ]:
import matplotlib.pyplot as plt

def plot_loss(history, epoch_interval, model_name):
    epochs = range(1, len(history.history['loss']) + 1, epoch_interval)
    train_loss = history.history['loss'][::epoch_interval]
    val_loss = history.history['val_loss'][::epoch_interval]

    plt.figure()
    plt.plot(epochs, train_loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title(f'{model_name} - Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig("MLP Training and Validation Loss", dpi=300, bbox_inches='tight')
    plt.show()
    
plot_loss(training_history, 5, 'MLP')

Plot a confusion matrix

In [ ]:
def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(8, 8))
    sns.heatmap(cm_normalized, annot=True, cmap='Blues', fmt='.2f', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('MLP Confusion Matrix')
    plt.savefig("MLP Confusion Matrix", dpi=300, bbox_inches='tight')
    plt.show()
    return(cm)
    
cm = plot_confusion_matrix(y_test_encoded, y_pred_classes, ['Weak', 'Medium', 'Strong', 'Very strong'])